In [1]:
!pip install -U spacy
import spacy
!python -m spacy download en
sp = spacy.load("en_core_web_sm")
import csv
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import os
!pip install nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from IPython.display import display, HTML
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install -U gensim
import gensim
from scipy import spatial
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
import re
from statistics import mean
import json
# to make our plot outputs appear and be stored within the notebook:
%matplotlib inline 
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 5)

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
output_df = pd.read_excel("output.xls")
text_df = pd.read_csv("18_093_attachment_text_Aug21.csv", encoding="ISO-8859-1")[["Date", "Attachment_text", "cn_doc_id", "Summary", "BRCID"]]

/tmp/ipykernel_2435392/2352958704.py:2: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

In [3]:
complete_data = pd.merge(output_df, text_df, left_on='CN_Doc_ID', right_on='cn_doc_id', how='right')
complete_data.sort_values("ethnicitycleaned", axis=0, inplace=True)

In [4]:
complete_data.rename(columns={"ethnicitycleaned": "Ethnicity", "Attachment_text": "Patient_Summary", "age": "Age", "Gender_ID": "Gender", "diagnosis_date": "Diagnosis_Date", "document_date": "Document_Date", "CN_Doc_ID": "unique_id"}, inplace=True)
complete_data = complete_data.dropna(subset=["Ethnicity", "Patient_Summary"])
# display(complete_data)
print(complete_data["Ethnicity"].value_counts())

Ethnicity
British (A)                    588
Any other ethnic group (S)     240
                              ... 
Chinese (R)                     14
White and Black African (E)     10
Name: count, Length: 16, dtype: int64


In [5]:
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other black background (P)", "Black").replace("African (N)", "Black").replace("Caribbean (M)", "Black").replace("White and Black African (E)", "Mixed Black").replace("White and black Caribbean (D)", "Mixed Black")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Indian (H)", "South Asian").replace("Pakistani (J)", "Asian").replace("Bangladeshi (K)", "Asian")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Chinese (R)", "Asian").replace("Any other Asian background (L)", "Asian")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other white background (C)", "White").replace("Irish (B)", "White").replace("British (A)", "White")
complete_data["Ethnicity"] = complete_data["Ethnicity"].replace("Any other mixed background (G)", "Other")
print(complete_data["Ethnicity"].value_counts())

Ethnicity
White          743
Black          393
              ... 
Mixed Black     31
Other           19
Name: count, Length: 8, dtype: int64


In [6]:
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("dr ".casefold(), "doctor ")
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("bipolar disorder".casefold(), "bipolar_disorder")
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("council housing".casefold(), "council_housing")
complete_data["Patient_Summary"] = complete_data["Patient_Summary"].replace("council flat".casefold(), "council_flat")

In [7]:
white_data = complete_data[complete_data["Ethnicity"] == "White"]
black_data = complete_data[complete_data["Ethnicity"] == "Black"]
mixed_data = complete_data[complete_data["Ethnicity"] == "Mixed Black"]
asian_data = complete_data[complete_data["Ethnicity"] == "Asian"]

print(white_data.size)
print(black_data.size)
print(mixed_data.size)
print(asian_data.size)

# display(white_data)
# display(black_data)
# display(asian_data)

10402
5502
434
1470


In [8]:
white_data['Patient_Summary_sp'] = [sp(white_data['Patient_Summary'].iloc[i]) for i in range(white_data.shape[0])]
black_data['Patient_Summary_sp'] = [sp(black_data['Patient_Summary'].iloc[i]) for i in range(black_data.shape[0])]
mixed_data['Patient_Summary_sp'] = [sp(mixed_data['Patient_Summary'].iloc[i]) for i in range(mixed_data.shape[0])]
asian_data['Patient_Summary_sp'] = [sp(asian_data['Patient_Summary'].iloc[i]) for i in range(asian_data.shape[0])]
print("Finished running spacy")

white_data['tokens'] = [[word.text for word in white_data['Patient_Summary_sp'].iloc[i]] for i in range(white_data.shape[0])]
black_data['tokens'] = [[word.text for word in black_data['Patient_Summary_sp'].iloc[i]] for i in range(black_data.shape[0])]
mixed_data['tokens'] = [[word.text for word in mixed_data['Patient_Summary_sp'].iloc[i]] for i in range(mixed_data.shape[0])]
asian_data['tokens'] = [[word.text for word in asian_data['Patient_Summary_sp'].iloc[i]] for i in range(asian_data.shape[0])]
print("Finished running tokens")
white_data['lemmas'] = [[word.lemma_ for word in white_data['Patient_Summary_sp'].iloc[i]] for i in range(white_data.shape[0])]
black_data['lemmas'] = [[word.lemma_ for word in black_data['Patient_Summary_sp'].iloc[i]] for i in range(black_data.shape[0])]
mixed_data['lemmas'] = [[word.lemma_ for word in mixed_data['Patient_Summary_sp'].iloc[i]] for i in range(mixed_data.shape[0])]
asian_data['lemmas'] = [[word.lemma_ for word in asian_data['Patient_Summary_sp'].iloc[i]] for i in range(asian_data.shape[0])]
print("Finished running lemmas")

Finished running spacy
Finished running tokens
Finished running lemmas


In [9]:
for i in range(white_data.shape[0]):
    white_data['lemmas'].iloc[i] = [t for t in white_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(black_data.shape[0]):
    black_data['lemmas'].iloc[i] = [t for t in black_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(mixed_data.shape[0]):
    mixed_data['lemmas'].iloc[i] = [t for t in mixed_data['lemmas'].iloc[i] if t.isalnum()]
for i in range(asian_data.shape[0]):
    asian_data['lemmas'].iloc[i] = [t for t in asian_data['lemmas'].iloc[i] if t.isalnum()]

In [10]:
stop_words = set(stopwords.words('english'))
white_data["lemmas_no_stop"] = ''
black_data['lemmas_no_stop'] = ''
mixed_data['lemmas_no_stop'] = ''
asian_data['lemmas_no_stop'] = ''
for i in range(white_data.shape[0]):
    white_data['lemmas_no_stop'].iloc[i] = [t for t in white_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(black_data.shape[0]):
    black_data['lemmas_no_stop'].iloc[i] = [t for t in black_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(mixed_data.shape[0]):
    mixed_data['lemmas_no_stop'].iloc[i] = [t for t in mixed_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]
for i in range(asian_data.shape[0]):
    asian_data['lemmas_no_stop'].iloc[i] = [t for t in asian_data['lemmas'].iloc[i] if t.lower() not in stop_words and not t.isnumeric() and t.lower() not in ["i", "the", "SLaM", "i'm", "i’m", "zzzzz", "qqqqq"]]

print("FINISHED REMOVING STOPWORDS")

FINISHED REMOVING STOPWORDS


In [11]:
white_data.to_csv(index=False)
black_data.to_csv(index=False)
mixed_data.to_csv(index=False)
asian_data.to_csv(index=False)

'brcid,Gender,dob,Ethnicity,Date_Of_Death,Age,Diagnosis_Date,Document_Date,unique_id,Date,Patient_Summary,cn_doc_id,Summary,BRCID,Patient_Summary_sp,tokens,lemmas,lemmas_no_stop\n10375735.0,Female,1963-07-01,Asian,,55.0,2019-04-02,2019-01-05,564647284060092,00:00.0,"            Psychological Medicine & Older Adults Directorate ZZZZZ  Integrated Mental Health of Older Adults Services ZZZZZ  Memory Service  Heavers Resource Centre  122 Selhurst Road  South Norwood  London SE25 6LL  Telephone Number: 020 3228 9500  e-mail: cms@slam.nhs.uk www.slam.nhs.uk \tDr A Tavares  \tStovell House Surgery  188 Lower Addiscombe Road Addiscombe ZZZZZ  ZZZZZ  CR0 6AH  \t \t   Dear Dr Tavares \tRe: Mrs ZZZZZ  ZZZZZ                                    \t      D.O.B: ZZZZZ  \tAge: 55  \tAddress: ZZZZZ ZZZZZ . ZZZZZ . ZZZZZ    \tNHS no: ZZZZZ     Mrs ZZZZZ  recently attended the Maudsley Hospital for an MRI brain scan as part of her memory assessment. On the whole the intracranial appearances are normal. How

In [12]:
def print_vocab(model, top_n = None):
    count = 0
    if top_n is not None:
        for index, word in enumerate(model.wv.index_to_key):
            count+= 1
            if count < top_n:
                print(f"WORD #{index}/{len(model.wv.index_to_key)} IS: {word}")

In [13]:
white_female_data = white_data[white_data["Gender"] == "Female"]
white_male_data = white_data[white_data["Gender"] == "Male"]

black_female_data = black_data[black_data["Gender"] == "Female"]
black_male_data = black_data[black_data["Gender"] == "Male"]

mixed_female_data = mixed_data[mixed_data["Gender"] == "Female"]
mixed_male_data = mixed_data[mixed_data["Gender"] == "Male"]

asian_female_data = asian_data[asian_data["Gender"] == "Female"]
asian_male_data = asian_data[asian_data["Gender"] == "Male"]

print(f"WHITE FEMALE: {white_female_data.size}")
print(f"BLACK FEMALE: {black_female_data.size}")
print(f"MIXED FEMALE: {mixed_female_data.size}")
print(f"ASIAN FEMALE: {asian_female_data.size}")

print(f"WHITE MALE: {white_male_data.size}")
print(f"BLACK MALE: {black_male_data.size}")
print(f"MIXED MALE: {mixed_male_data.size}")
print(f"ASIAN MALE: {asian_male_data.size}")

WHITE FEMALE: 9000
BLACK FEMALE: 5544
MIXED FEMALE: 450
ASIAN FEMALE: 1260
WHITE MALE: 4374
BLACK MALE: 1530
MIXED MALE: 108
ASIAN MALE: 630


## CBOW Model Calculations & Saves

In [ ]:
white_fem_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---WHITE FEMALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---WHITE FEMALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---WHITE FEMALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---WHITE FEMALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
white_fem_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---WHITE FEMALE")

In [ ]:
white_mal_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---WHITE MALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---WHITE MALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---WHITE MALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---WHITE MALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
white_mal_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---WHITE MALE")

In [ ]:
# white cbow save & calculate avg
synonyms = dict({"medication": "medicine",
                 "therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "mood": "temper",
                 "information": "info",
                 "treatment": "intervention",
                 "social": "sociable",
                 "anxious": "nervous",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "aware": "mindful",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })

white_fem_cbow_models_synonymity_average = {}
white_fem_cbow_names = list(white_fem_cbow_models.keys())
white_fem_cbow_vals = list(white_fem_cbow_models.values())
for i in range(len(white_fem_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(white_fem_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    white_fem_cbow_models_synonymity_average[white_fem_cbow_names[i]] = average_synonimity

white_fem_cbow_model_chosen = max(white_fem_cbow_models_synonymity_average, key=white_fem_cbow_models_synonymity_average.get)
print(f"WHITE FEMALE CBOW MODEL: {white_fem_cbow_model_chosen}")
print(f"\tSYNONIMITY: {white_fem_cbow_models_synonymity_average[white_fem_cbow_model_chosen]}")
white_fem_cbow_models[white_fem_cbow_model_chosen].save(os.path.join('white_models', white_fem_cbow_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "simple": "easy",
                 "appointment": "engagement",
                 "road": "route",
                 "history": "account",
                 "mood": "temper",
                 "information": "info",
                 "treatment": "intervention",
                 "anxious": "nervous",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "aware": "mindful",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })
white_mal_cbow_models_synonymity_average = {}
white_mal_cbow_names = list(white_mal_cbow_models.keys())
white_mal_cbow_vals = list(white_mal_cbow_models.values())
for i in range(len(white_mal_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(white_mal_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    white_mal_cbow_models_synonymity_average[white_mal_cbow_names[i]] = average_synonimity

white_mal_cbow_model_chosen = max(white_mal_cbow_models_synonymity_average, key=white_mal_cbow_models_synonymity_average.get)
print(f"WHITE MALE CBOW MODEL: {white_mal_cbow_model_chosen}")
print(f"\tSYNONIMITY: {white_mal_cbow_models_synonymity_average[white_mal_cbow_model_chosen]}")
white_mal_cbow_models[white_mal_cbow_model_chosen].save(os.path.join('white_models', white_mal_cbow_model_chosen))

In [ ]:
black_fem_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---BLACK FEMALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---BLACK FEMALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---BLACK FEMALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---BLACK FEMALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0

cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
black_fem_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20

cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1

cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---BLACK FEMALE")

In [ ]:
black_mal_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---BLACK MALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---BLACK MALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---BLACK MALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---BLACK MALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
black_mal_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20
cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---BLACK MALE")

In [ ]:
# black cbow save & calculate avg
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })

black_fem_cbow_models_synonymity_average = {}
black_fem_cbow_names = list(black_fem_cbow_models.keys())
black_fem_cbow_vals = list(black_fem_cbow_models.values())
for i in range(len(black_fem_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(black_fem_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    black_fem_cbow_models_synonymity_average[black_fem_cbow_names[i]] = average_synonimity

# for i in black_fem_cbow_models_synonymity_average.keys():
#     print(f"{i}: {black_fem_cbow_models_synonymity_average[i]}")
black_fem_cbow_model_chosen = max(black_fem_cbow_models_synonymity_average, key=black_fem_cbow_models_synonymity_average.get)
print(f"BLACK FEMALE CBOW MODEL: {black_fem_cbow_model_chosen}")
print(f"\tSYNONIMITY: {black_fem_cbow_models_synonymity_average[black_fem_cbow_model_chosen]}")
black_fem_cbow_models[black_fem_cbow_model_chosen].save(os.path.join('black_models', black_fem_cbow_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "treatment": "intervention",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "appointment": "date"
                })
black_mal_cbow_models_synonymity_average = {}
black_mal_cbow_names = list(black_mal_cbow_models.keys())
black_mal_cbow_vals = list(black_mal_cbow_models.values())
for i in range(len(black_mal_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(black_mal_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    black_mal_cbow_models_synonymity_average[black_mal_cbow_names[i]] = average_synonimity

black_mal_cbow_model_chosen = max(black_mal_cbow_models_synonymity_average, key=black_mal_cbow_models_synonymity_average.get)
print(f"BLACK MALE CBOW MODEL: {black_mal_cbow_model_chosen}")
print(f"\tSYNONIMITY: {black_mal_cbow_models_synonymity_average[black_mal_cbow_model_chosen]}")
black_mal_cbow_models[black_mal_cbow_model_chosen].save(os.path.join('black_models', black_mal_cbow_model_chosen))

In [ ]:
mixed_fem_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---MIXED FEMALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---MIXED FEMALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---MIXED FEMALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---MIXED FEMALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
mixed_fem_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20
cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---MIXED FEMALE")

In [ ]:
mixed_mal_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---MIXED MALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---MIXED MALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---MIXED MALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---MIXED MALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
mixed_mal_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20
cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---MIXED MALE")

In [ ]:
# mixed cbow save & calculate avg
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "treatment": "intervention",
                 "appointment": "date"
                })

mixed_fem_cbow_models_synonymity_average = {}
mixed_fem_cbow_names = list(mixed_fem_cbow_models.keys())
mixed_fem_cbow_vals = list(mixed_fem_cbow_models.values())
for i in range(len(mixed_fem_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(mixed_fem_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    mixed_fem_cbow_models_synonymity_average[mixed_fem_cbow_names[i]] = average_synonimity

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "appointment": "date"
                })
mixed_fem_cbow_model_chosen = max(mixed_fem_cbow_models_synonymity_average, key=mixed_fem_cbow_models_synonymity_average.get)
print(f"MIXED FEMALE CBOW MODEL: {mixed_fem_cbow_model_chosen}")
print(f"\tSYNONIMITY: {mixed_fem_cbow_models_synonymity_average[mixed_fem_cbow_model_chosen]}")
mixed_fem_cbow_models[mixed_fem_cbow_model_chosen].save(os.path.join('mixed_models', mixed_fem_cbow_model_chosen))

mixed_mal_cbow_models_synonymity_average = {}
mixed_mal_cbow_names = list(mixed_mal_cbow_models.keys())
mixed_mal_cbow_vals = list(mixed_mal_cbow_models.values())
for i in range(len(mixed_mal_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(mixed_mal_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    mixed_mal_cbow_models_synonymity_average[mixed_mal_cbow_names[i]] = average_synonimity

mixed_mal_cbow_model_chosen = max(mixed_mal_cbow_models_synonymity_average, key=mixed_mal_cbow_models_synonymity_average.get)
print(f"MIXED MALE CBOW MODEL: {mixed_mal_cbow_model_chosen}")
print(f"\tSYNONIMITY: {mixed_mal_cbow_models_synonymity_average[mixed_mal_cbow_model_chosen]}")
mixed_mal_cbow_models[mixed_mal_cbow_model_chosen].save(os.path.join('mixed_models', mixed_mal_cbow_model_chosen))

In [ ]:
asian_fem_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---ASIAN FEMALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---ASIAN FEMALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---ASIAN FEMALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---ASIAN FEMALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
asian_fem_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20
cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---ASIAN FEMALE")

In [ ]:
asian_mal_cbow_models = {}
# 100 dimensions
cbow_w3_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_100"] = cbow_w3_f1_100
cbow_w5_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_100"] = cbow_w5_f1_100
cbow_w10_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100"] = cbow_w10_f1_100
cbow_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_100_mc0"] = cbow_w3_f1_100_mc0
cbow_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_100_mc0"] = cbow_w5_f1_100_mc0
cbow_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_mc0"] = cbow_w10_f1_100_mc0
cbow_w10_f1_100_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_mc2"] = cbow_w10_f1_100_mc2
cbow_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_neg2"] = cbow_w10_f1_100_neg2
cbow_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_neg5"] = cbow_w10_f1_100_neg5
cbow_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_ns_half_neg2"] = cbow_w10_f1_100_ns_half_neg2
cbow_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_ns_half_neg3"] = cbow_w10_f1_100_ns_half_neg3
cbow_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_ns_half_neg5"] = cbow_w10_f1_100_ns_half_neg5
cbow_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_neg2_ws2"] = cbow_w10_f1_100_neg2_ws2
cbow_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_100_neg5_ws5"] = cbow_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION MODELS---ASIAN MALE")

# 300 dimensions
cbow_w3_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_300"] = cbow_w3_f1_300
cbow_w5_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_300"] = cbow_w5_f1_300
cbow_w10_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300"] = cbow_w10_f1_300
cbow_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_300_mc0"] = cbow_w3_f1_300_mc0
cbow_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_300_mc0"] = cbow_w5_f1_300_mc0
cbow_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_mc0"] = cbow_w10_f1_300_mc0
cbow_w10_f1_300_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_mc2"] = cbow_w10_f1_300_mc2
cbow_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_neg2"] = cbow_w10_f1_300_neg2
cbow_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_neg5"] = cbow_w10_f1_300_neg5
cbow_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_ns_half_neg2"] = cbow_w10_f1_300_ns_half_neg2
cbow_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_ns_half_neg3"] = cbow_w10_f1_300_ns_half_neg3
cbow_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_ns_half_neg5"] = cbow_w10_f1_300_ns_half_neg5
cbow_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_neg2_ws2"] = cbow_w10_f1_300_neg2_ws2
cbow_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_300_neg5_ws5"] = cbow_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION MODELS---ASIAN MALE")

# 500 dimensions
cbow_w3_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_500"] = cbow_w3_f1_500
cbow_w5_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_500"] = cbow_w5_f1_500
cbow_w10_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500"] = cbow_w10_f1_500
cbow_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_500_mc0"] = cbow_w3_f1_500_mc0
cbow_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_500_mc0"] = cbow_w5_f1_500_mc0
cbow_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_mc0"] = cbow_w10_f1_500_mc0
cbow_w10_f1_500_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_mc2"] = cbow_w10_f1_500_mc2
cbow_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_neg2"] = cbow_w10_f1_500_neg2
cbow_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_neg5"] = cbow_w10_f1_500_neg5
cbow_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_ns_half_neg2"] = cbow_w10_f1_500_ns_half_neg2
cbow_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_ns_half_neg3"] = cbow_w10_f1_500_ns_half_neg3
cbow_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_ns_half_neg5"] = cbow_w10_f1_500_ns_half_neg5
cbow_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_neg2_ws2"] = cbow_w10_f1_500_neg2_ws2
cbow_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_500_neg5_ws5"] = cbow_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION MODELS---ASIAN MALE")

# 700 dimensions
cbow_w3_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_700"] = cbow_w3_f1_700
cbow_w5_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_700"] = cbow_w5_f1_700
cbow_w10_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700"] = cbow_w10_f1_700
cbow_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_700_mc0"] = cbow_w3_f1_700_mc0
cbow_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_700_mc0"] = cbow_w5_f1_700_mc0
cbow_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_mc0"] = cbow_w10_f1_700_mc0
cbow_w10_f1_700_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_mc2"] = cbow_w10_f1_700_mc2
cbow_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_neg2"] = cbow_w10_f1_700_neg2
cbow_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_neg5"] = cbow_w10_f1_700_neg5
cbow_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_ns_half_neg2"] = cbow_w10_f1_700_ns_half_neg2
cbow_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_ns_half_neg3"] = cbow_w10_f1_700_ns_half_neg3
cbow_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_ns_half_neg5"] = cbow_w10_f1_700_ns_half_neg5
cbow_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_neg2_ws2"] = cbow_w10_f1_700_neg2_ws2
cbow_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_700_neg5_ws5"] = cbow_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION MODELS---ASIAN MALE")

# 1000 dimensions
cbow_w3_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_1000"] = cbow_w3_f1_1000
cbow_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w2_f1_1000_ns_half_neg5"] = cbow_w2_f1_1000_ns_half_neg5
cbow_w5_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_1000"] = cbow_w5_f1_1000
cbow_w10_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000"] = cbow_w10_f1_1000
cbow_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_1000_mc0"] = cbow_w3_f1_1000_mc0
cbow_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 0, epochs = 20)
asian_mal_cbow_models["cbow_w3_f1_1000_mc0_e20"] = cbow_w3_f1_1000_mc0_e20
cbow_w3_f1_1000_mc1 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w3_f1_1000_mc1"] = cbow_w3_f1_1000_mc1
cbow_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w5_f1_1000_mc0"] = cbow_w5_f1_1000_mc0
cbow_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_mc0"] = cbow_w10_f1_1000_mc0
cbow_w10_f1_1000_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 0, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_mc2"] = cbow_w10_f1_1000_mc2
cbow_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_neg2"] = cbow_w10_f1_1000_neg2
cbow_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_neg5"] = cbow_w10_f1_1000_neg5
cbow_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg2"] = cbow_w10_f1_1000_ns_half_neg2
cbow_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg3"] = cbow_w10_f1_1000_ns_half_neg3
cbow_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 0, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_ns_half_neg5"] = cbow_w10_f1_1000_ns_half_neg5
cbow_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 0, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_neg2_ws2"] = cbow_w10_f1_1000_neg2_ws2
cbow_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 0, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_cbow_models["cbow_w10_f1_1000_neg5_ws5"] = cbow_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION MODELS---ASIAN MALE")

In [ ]:
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "appointment": "date"
                })

asian_fem_cbow_models_synonymity_average = {}
asian_fem_cbow_names = list(asian_fem_cbow_models.keys())
asian_fem_cbow_vals = list(asian_fem_cbow_models.values())
for i in range(len(asian_fem_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(asian_fem_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    asian_fem_cbow_models_synonymity_average[asian_fem_cbow_names[i]] = average_synonimity

asian_fem_cbow_model_chosen = max(asian_fem_cbow_models_synonymity_average, key=asian_fem_cbow_models_synonymity_average.get)
print(f"ASIAN FEMALE CBOW MODEL: {asian_fem_cbow_model_chosen}")
print(f"\tSYNONIMITY: {asian_fem_cbow_models_synonymity_average[asian_fem_cbow_model_chosen]}")
asian_fem_cbow_models[asian_fem_cbow_model_chosen].save(os.path.join('asian_models', asian_fem_cbow_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "treatment": "intervention",
                 "difficult": "hard",
                 "appointment": "date"
                })
asian_mal_cbow_models_synonymity_average = {}
asian_mal_cbow_names = list(asian_mal_cbow_models.keys())
asian_mal_cbow_vals = list(asian_mal_cbow_models.values())
for i in range(len(asian_mal_cbow_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(asian_mal_cbow_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    asian_mal_cbow_models_synonymity_average[asian_mal_cbow_names[i]] = average_synonimity

asian_mal_cbow_model_chosen = max(asian_mal_cbow_models_synonymity_average, key=asian_mal_cbow_models_synonymity_average.get)
print(f"ASIAN MALE CBOW MODEL: {asian_mal_cbow_model_chosen}")
print(f"\tSYNONIMITY: {asian_mal_cbow_models_synonymity_average[asian_mal_cbow_model_chosen]}")
asian_mal_cbow_models[asian_mal_cbow_model_chosen].save(os.path.join('asian_models', asian_mal_cbow_model_chosen))

## Skip-Gram Model Calculations

In [14]:
white_fem_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---WHITE FEMALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---WHITE FEMALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---WHITE FEMALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---WHITE FEMALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0

skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
white_fem_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20

skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_fem_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1

skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
white_fem_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_fem_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---WHITE FEMALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---WHITE FEMALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---WHITE FEMALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---WHITE FEMALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---WHITE FEMALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---WHITE FEMALE


In [15]:
white_mal_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---WHITE MALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---WHITE MALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---WHITE MALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---WHITE MALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0
skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
white_mal_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20
skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
white_mal_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1
skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
white_mal_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(white_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
white_mal_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---WHITE MALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---WHITE MALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---WHITE MALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---WHITE MALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---WHITE MALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---WHITE MALE


In [16]:
# white skip save & calculate avg
synonyms = dict({"medication": "medicine",
                 "therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "mood": "temper",
                 "information": "info",
                 "treatment": "intervention",
                 "social": "sociable",
                 "anxious": "nervous",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "aware": "mindful",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })

white_fem_skip_models_synonymity_average = {}
white_fem_skip_names = list(white_fem_skip_models.keys())
white_fem_skip_vals = list(white_fem_skip_models.values())
for i in range(len(white_fem_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(white_fem_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    white_fem_skip_models_synonymity_average[white_fem_skip_names[i]] = average_synonimity

white_fem_skip_model_chosen = max(white_fem_skip_models_synonymity_average, key=white_fem_skip_models_synonymity_average.get)
print(f"WHITE FEMALE SKIP MODEL: {white_fem_skip_model_chosen}")
print(f"\tSYNONIMITY: {white_fem_skip_models_synonymity_average[white_fem_skip_model_chosen]}")
white_fem_skip_models[white_fem_skip_model_chosen].save(os.path.join('white_models', white_fem_skip_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "simple": "easy",
                 "appointment": "engagement",
                 "road": "route",
                 "history": "account",
                 "mood": "temper",
                 "information": "info",
                 "treatment": "intervention",
                 "anxious": "nervous",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "aware": "mindful",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })
white_mal_skip_models_synonymity_average = {}
white_mal_skip_names = list(white_mal_skip_models.keys())
white_mal_skip_vals = list(white_mal_skip_models.values())
for i in range(len(white_mal_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(white_mal_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    white_mal_skip_models_synonymity_average[white_mal_skip_names[i]] = average_synonimity

white_mal_skip_model_chosen = max(white_mal_skip_models_synonymity_average, key=white_mal_skip_models_synonymity_average.get)
print(f"WHITE MALE SKIP MODEL: {white_mal_skip_model_chosen}")
print(f"\tSYNONIMITY: {white_mal_skip_models_synonymity_average[white_mal_skip_model_chosen]}")
white_mal_skip_models[white_mal_skip_model_chosen].save(os.path.join('white_models', white_mal_skip_model_chosen))

WHITE FEMALE SKIP MODEL: skip_w10_f1_700_neg2_ws2
	SYNONIMITY: 0.8104930520057678
WHITE MALE SKIP MODEL: skip_w3_f1_1000_mc1
	SYNONIMITY: 0.9079453945159912


In [17]:
black_fem_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0

skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
black_fem_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20

skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_fem_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1

skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
black_fem_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_fem_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE


In [18]:
black_mal_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK MALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK MALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK MALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK MALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0

skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
black_mal_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20

skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
black_mal_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1

skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
black_mal_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(black_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
black_mal_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK MALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK MALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK MALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK MALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK MALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK MALE


In [19]:
# black skip save & calculate avg
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "confidential": "private",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "clinician": "doctor",
                 "clinician": "therapist",
                 "appointment": "date"
                })

black_fem_skip_models_synonymity_average = {}
black_fem_skip_names = list(black_fem_skip_models.keys())
black_fem_skip_vals = list(black_fem_skip_models.values())
for i in range(len(black_fem_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(black_fem_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    black_fem_skip_models_synonymity_average[black_fem_skip_names[i]] = average_synonimity

# for i in black_fem_skip_models_synonymity_average.keys():
#     print(f"{i}: {black_fem_skip_models_synonymity_average[i]}")
black_fem_skip_model_chosen = max(black_fem_skip_models_synonymity_average, key=black_fem_skip_models_synonymity_average.get)
print(f"BLACK FEMALE SKIP MODEL: {black_fem_skip_model_chosen}")
print(f"\tSYNONIMITY: {black_fem_skip_models_synonymity_average[black_fem_skip_model_chosen]}")
black_fem_skip_models[black_fem_skip_model_chosen].save(os.path.join('black_models', black_fem_skip_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "treatment": "intervention",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "appointment": "date"
                })
black_mal_skip_models_synonymity_average = {}
black_mal_skip_names = list(black_mal_skip_models.keys())
black_mal_skip_vals = list(black_mal_skip_models.values())
for i in range(len(black_mal_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(black_mal_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    black_mal_skip_models_synonymity_average[black_mal_skip_names[i]] = average_synonimity

black_mal_skip_model_chosen = max(black_mal_skip_models_synonymity_average, key=black_mal_skip_models_synonymity_average.get)
print(f"BLACK MALE SKIP MODEL: {black_mal_skip_model_chosen}")
print(f"\tSYNONIMITY: {black_mal_skip_models_synonymity_average[black_mal_skip_model_chosen]}")
black_mal_skip_models[black_mal_skip_model_chosen].save(os.path.join('black_models', black_mal_skip_model_chosen))

BLACK FEMALE SKIP MODEL: skip_w2_f1_1000_ns_half_neg5
	SYNONIMITY: 0.872725784778595
BLACK MALE SKIP MODEL: skip_w3_f1_1000_mc1
	SYNONIMITY: 0.9990812540054321


In [20]:
mixed_fem_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0

skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
mixed_fem_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20

skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1

skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_fem_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE


In [21]:
mixed_mal_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0
skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
mixed_mal_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20
skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1
skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(mixed_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
mixed_mal_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---BLACK FEMALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---BLACK FEMALE


In [22]:
# mixed skip save & calculate avg
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "treatment": "intervention",
                 "appointment": "date"
                })

mixed_fem_skip_models_synonymity_average = {}
mixed_fem_skip_names = list(mixed_fem_skip_models.keys())
mixed_fem_skip_vals = list(mixed_fem_skip_models.values())
for i in range(len(mixed_fem_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(mixed_fem_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    mixed_fem_skip_models_synonymity_average[mixed_fem_skip_names[i]] = average_synonimity

# for i in mixed_fem_skip_models_synonymity_average.keys():
#     print(f"{i}: {mixed_fem_skip_models_synonymity_average[i]}")
mixed_fem_skip_model_chosen = max(mixed_fem_skip_models_synonymity_average, key=mixed_fem_skip_models_synonymity_average.get)
print(f"MIXED FEMALE SKIP MODEL: {mixed_fem_skip_model_chosen}")
print(f"\tSYNONIMITY: {mixed_fem_skip_models_synonymity_average[mixed_fem_skip_model_chosen]}")
mixed_fem_skip_models[mixed_fem_skip_model_chosen].save(os.path.join('mixed_models', mixed_fem_skip_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "appointment": "date"
                })
mixed_mal_skip_models_synonymity_average = {}
mixed_mal_skip_names = list(mixed_mal_skip_models.keys())
mixed_mal_skip_vals = list(mixed_mal_skip_models.values())
for i in range(len(mixed_mal_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(mixed_mal_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    mixed_mal_skip_models_synonymity_average[mixed_mal_skip_names[i]] = average_synonimity

mixed_mal_skip_model_chosen = max(mixed_mal_skip_models_synonymity_average, key=mixed_mal_skip_models_synonymity_average.get)
print(f"MIXED MALE SKIP MODEL: {mixed_mal_skip_model_chosen}")
print(f"\tSYNONIMITY: {mixed_mal_skip_models_synonymity_average[mixed_mal_skip_model_chosen]}")
mixed_mal_skip_models[mixed_mal_skip_model_chosen].save(os.path.join('mixed_models', mixed_mal_skip_model_chosen))

MIXED FEMALE SKIP MODEL: skip_w3_f1_1000_mc0_e20
	SYNONIMITY: 0.9993669986724854
MIXED MALE SKIP MODEL: skip_w10_f1_1000_ns_half_neg5
	SYNONIMITY: 0.9996933341026306


In [23]:
asian_fem_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0
skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
asian_fem_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20
skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1
skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_female_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_fem_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---ASIAN FEMALE


In [24]:
asian_mal_skip_models = {}
# 100 dimensions
skip_w3_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_100"] = skip_w3_f1_100
skip_w5_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_100"] = skip_w5_f1_100
skip_w10_f1_100 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100"] = skip_w10_f1_100
skip_w3_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_100_mc0"] = skip_w3_f1_100_mc0
skip_w5_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_100_mc0"] = skip_w5_f1_100_mc0
skip_w10_f1_100_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_mc0"] = skip_w10_f1_100_mc0
skip_w10_f1_100_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=100, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_mc2"] = skip_w10_f1_100_mc2
skip_w10_f1_100_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_neg2"] = skip_w10_f1_100_neg2
skip_w10_f1_100_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_neg5"] = skip_w10_f1_100_neg5
skip_w10_f1_100_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_ns_half_neg2"] = skip_w10_f1_100_ns_half_neg2
skip_w10_f1_100_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_ns_half_neg3"] = skip_w10_f1_100_ns_half_neg3
skip_w10_f1_100_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_ns_half_neg5"] = skip_w10_f1_100_ns_half_neg5
skip_w10_f1_100_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_neg2_ws2"] = skip_w10_f1_100_neg2_ws2
skip_w10_f1_100_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=100, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_100_neg5_ws5"] = skip_w10_f1_100_neg5_ws5
print("TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---ASIAN MALE")

# 300 dimensions
skip_w3_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_300"] = skip_w3_f1_300
skip_w5_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_300"] = skip_w5_f1_300
skip_w10_f1_300 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300"] = skip_w10_f1_300
skip_w3_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_300_mc0"] = skip_w3_f1_300_mc0
skip_w5_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_300_mc0"] = skip_w5_f1_300_mc0
skip_w10_f1_300_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_mc0"] = skip_w10_f1_300_mc0
skip_w10_f1_300_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=300, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_mc2"] = skip_w10_f1_300_mc2
skip_w10_f1_300_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_neg2"] = skip_w10_f1_300_neg2
skip_w10_f1_300_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_neg5"] = skip_w10_f1_300_neg5
skip_w10_f1_300_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_ns_half_neg2"] = skip_w10_f1_300_ns_half_neg2
skip_w10_f1_300_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_ns_half_neg3"] = skip_w10_f1_300_ns_half_neg3
skip_w10_f1_300_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_ns_half_neg5"] = skip_w10_f1_300_ns_half_neg5
skip_w10_f1_300_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_neg2_ws2"] = skip_w10_f1_300_neg2_ws2
skip_w10_f1_300_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=300, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_300_neg5_ws5"] = skip_w10_f1_300_neg5_ws5
print("TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---ASIAN MALE")

# 500 dimensions
skip_w3_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_500"] = skip_w3_f1_500
skip_w5_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_500"] = skip_w5_f1_500
skip_w10_f1_500 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500"] = skip_w10_f1_500
skip_w3_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_500_mc0"] = skip_w3_f1_500_mc0
skip_w5_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_500_mc0"] = skip_w5_f1_500_mc0
skip_w10_f1_500_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_mc0"] = skip_w10_f1_500_mc0
skip_w10_f1_500_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=500, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_mc2"] = skip_w10_f1_500_mc2
skip_w10_f1_500_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_neg2"] = skip_w10_f1_500_neg2
skip_w10_f1_500_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_neg5"] = skip_w10_f1_500_neg5
skip_w10_f1_500_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_ns_half_neg2"] = skip_w10_f1_500_ns_half_neg2
skip_w10_f1_500_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_ns_half_neg3"] = skip_w10_f1_500_ns_half_neg3
skip_w10_f1_500_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_ns_half_neg5"] = skip_w10_f1_500_ns_half_neg5
skip_w10_f1_500_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_neg2_ws2"] = skip_w10_f1_500_neg2_ws2
skip_w10_f1_500_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=500, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_500_neg5_ws5"] = skip_w10_f1_500_neg5_ws5
print("TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---ASIAN MALE")

# 700 dimensions
skip_w3_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_700"] = skip_w3_f1_700
skip_w5_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_700"] = skip_w5_f1_700
skip_w10_f1_700 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700"] = skip_w10_f1_700
skip_w3_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_700_mc0"] = skip_w3_f1_700_mc0
skip_w5_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_700_mc0"] = skip_w5_f1_700_mc0
skip_w10_f1_700_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_mc0"] = skip_w10_f1_700_mc0
skip_w10_f1_700_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=700, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_mc2"] = skip_w10_f1_700_mc2
skip_w10_f1_700_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_neg2"] = skip_w10_f1_700_neg2
skip_w10_f1_700_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_neg5"] = skip_w10_f1_700_neg5
skip_w10_f1_700_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_ns_half_neg2"] = skip_w10_f1_700_ns_half_neg2
skip_w10_f1_700_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_ns_half_neg3"] = skip_w10_f1_700_ns_half_neg3
skip_w10_f1_700_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_ns_half_neg5"] = skip_w10_f1_700_ns_half_neg5
skip_w10_f1_700_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_neg2_ws2"] = skip_w10_f1_700_neg2_ws2
skip_w10_f1_700_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=700, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_700_neg5_ws5"] = skip_w10_f1_700_neg5_ws5
print("TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---ASIAN MALE")

# 1000 dimensions
skip_w3_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_1000"] = skip_w3_f1_1000
skip_w2_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w2_f1_1000_ns_half_neg5"] = skip_w2_f1_1000_ns_half_neg5
skip_w5_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_1000"] = skip_w5_f1_1000
skip_w10_f1_1000 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000"] = skip_w10_f1_1000
skip_w3_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_1000_mc0"] = skip_w3_f1_1000_mc0

skip_w3_f1_1000_mc0_e20 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 2, sg = 1, epochs = 20)
asian_mal_skip_models["skip_w3_f1_1000_mc0_e20"] = skip_w3_f1_1000_mc0_e20

skip_w3_f1_1000_mc1 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w3_f1_1000_mc1"] = skip_w3_f1_1000_mc1

skip_w5_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 3, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w5_f1_1000_mc0"] = skip_w5_f1_1000_mc0
skip_w10_f1_1000_mc0 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=0, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_mc0"] = skip_w10_f1_1000_mc0
skip_w10_f1_1000_mc2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=2, vector_size=1000, window = 10, sg = 1, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_mc2"] = skip_w10_f1_1000_mc2
skip_w10_f1_1000_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_neg2"] = skip_w10_f1_1000_neg2
skip_w10_f1_1000_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_neg5"] = skip_w10_f1_1000_neg5
skip_w10_f1_1000_ns_half_neg2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=2, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_ns_half_neg2"] = skip_w10_f1_1000_ns_half_neg2
skip_w10_f1_1000_ns_half_neg3 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=3, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_ns_half_neg3"] = skip_w10_f1_1000_ns_half_neg3
skip_w10_f1_1000_ns_half_neg5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 10, sg = 1, negative=5, ns_exponent=0.5, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_ns_half_neg5"] = skip_w10_f1_1000_ns_half_neg5
skip_w10_f1_1000_neg2_ws2 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 2, sg = 1, negative=2, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_neg2_ws2"] = skip_w10_f1_1000_neg2_ws2
skip_w10_f1_1000_neg5_ws5 = gensim.models.Word2Vec(asian_male_data['lemmas_no_stop'], min_count=1, vector_size=1000, window = 3, sg = 1, negative=5, ns_exponent=0.75, epochs = 10)
asian_mal_skip_models["skip_w10_f1_1000_neg5_ws5"] = skip_w10_f1_1000_neg5_ws5
print("TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---ASIAN MALE")

TRAINED ALL 100-DIMENSION SKIPGRAM MODELS---ASIAN MALE
TRAINED ALL 300-DIMENSION SKIPGRAM MODELS---ASIAN MALE
TRAINED ALL 500-DIMENSION SKIPGRAM MODELS---ASIAN MALE
TRAINED ALL 700-DIMENSION SKIPGRAM MODELS---ASIAN MALE
TRAINED ALL 1000-DIMENSION SKIPGRAM MODELS---ASIAN MALE


In [25]:
# asian skip save & calculate avg
synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "family": "household",
                 "history": "account",
                 "treatment": "intervention",
                 "feeling": "belief",
                 "feeling": "opinion",
                 "feeling": "impression",
                 "difficult": "hard",
                 "appointment": "date"
                })

asian_fem_skip_models_synonymity_average = {}
asian_fem_skip_names = list(asian_fem_skip_models.keys())
asian_fem_skip_vals = list(asian_fem_skip_models.values())
for i in range(len(asian_fem_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(asian_fem_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    asian_fem_skip_models_synonymity_average[asian_fem_skip_names[i]] = average_synonimity

# for i in asian_fem_skip_models_synonymity_average.keys():
#     print(f"{i}: {asian_fem_skip_models_synonymity_average[i]}")
asian_fem_skip_model_chosen = max(asian_fem_skip_models_synonymity_average, key=asian_fem_skip_models_synonymity_average.get)
print(f"ASIAN FEMALE SKIP MODEL: {asian_fem_skip_model_chosen}")
print(f"\tSYNONIMITY: {asian_fem_skip_models_synonymity_average[asian_fem_skip_model_chosen]}")
asian_fem_skip_models[asian_fem_skip_model_chosen].save(os.path.join('asian_models', asian_fem_skip_model_chosen))

synonyms = dict({"therapy": "treatment",
                 "appointment": "engagement",
                 "history": "account",
                 "treatment": "intervention",
                 "difficult": "hard",
                 "appointment": "date"
                })
asian_mal_skip_models_synonymity_average = {}
asian_mal_skip_names = list(asian_mal_skip_models.keys())
asian_mal_skip_vals = list(asian_mal_skip_models.values())
for i in range(len(asian_mal_skip_names)):
    average_synonimity = 0
    synonymities = list()
    for s1 in synonyms:
        synonymities.append(asian_mal_skip_vals[i].wv.similarity(s1, synonyms[s1]))
    average_synonimity = mean(synonymities)
    asian_mal_skip_models_synonymity_average[asian_mal_skip_names[i]] = average_synonimity

asian_mal_skip_model_chosen = max(asian_mal_skip_models_synonymity_average, key=asian_mal_skip_models_synonymity_average.get)
print(f"ASIAN MALE SKIP MODEL: {asian_mal_skip_model_chosen}")
print(f"\tSYNONIMITY: {asian_mal_skip_models_synonymity_average[asian_mal_skip_model_chosen]}")
asian_mal_skip_models[asian_mal_skip_model_chosen].save(os.path.join('asian_models', asian_mal_skip_model_chosen))

ASIAN FEMALE SKIP MODEL: skip_w2_f1_1000_ns_half_neg5
	SYNONIMITY: 0.9993869066238403
ASIAN MALE SKIP MODEL: skip_w5_f1_1000
	SYNONIMITY: 0.9989318251609802


## Embedding Calculations---CBOW

In [26]:
# GROUP 1 VOCAB: Symptoms
symptoms = ['delusional', 'manic', 'suicidal', 'delusions', 'mania', 'suicide', 'disordered', 'eating', 'insomnia', 'addiction', 'addicted', 'weight', 'paranoid', 'paranoia', 'hypochondria', 'hypochondriac']

# GROUP 2 VOCAB: Conditions
conditions = ['pregnant', 'pregnancy', 'schizophrenia', 'depression', 'anxiety', 'schizophrenic', 'depressed', 'anxious', 'bipolar_disorder', 'bipolar', 'autism', 'autistic', 'handicapped', 'handicap', 'learning', 'diabetes', 'obese', 'obesity', 'alcoholism', 'alcohol', 'medication', 'rape']

# GROUP 3 VOCAB: Interpersonal
interpersonal = ['family', 'relationship', 'mother', 'father', 'husband', 'wife', 'son', 'daughter', 'child', 'parent', 'parents', 'friends', 'support', 'marriage', 'partner', 'childcare']

# GROUP 4 VOCAB: Behaviour
behaviour = ['mood', 'behaviour', 'violent', 'agitated', 'moody', 'withdrawn', 'aggressive', 'submissive', 'quiet', 'speech', 'listen', 'sleep', 'irritable', 'beliefs']

# GROUP 5 VOCAB: Clinical
clinical = ['appointment', 'assessment', 'cancel', 'reschedule', 'treatment', 'schedule', 'intervention', 'medication', 'medicate', 'medicine', 'doctor', 'therapist', 'clinician']

# GROUP 6 VOCAB: Stereotypes
stereotypes = ['articulate', 'english', 'native', 'foreign', 'immigrant', 'immigration', 'visa', 'ethnic', 'cultural', 'culture']

# GROUP 7 VOCAB: Subjective
subjective = ['stubborn', 'frustrating', 'defiant', 'obstinate', 'unwilling', 'risk', 'personality', 'opinion', 'believe', 'seem', 'claim', 'think', 'recommend']

# GROUP 8 VOCAB: Economics
economics = ['car', 'house', 'ownership', 'wealth', 'poverty', 'impoverished', 'poor', 'council', 'employment', 'unemployed', 'job', 'profession']

In [ ]:
def find_embeddings(word_embeddings, word_list, model_list, chosen_name, ethnic_group, gender, num_words):
    for word in word_list:
        word_file = open(f"embeddings_final/{word}{num_words}.txt", "a")
        try:
            key = f"{word}_{ethnic_group}_{gender.lower()}_cbow"
            word_embeddings[key] = []
            word_file.write(f"---------{ethnic_group.upper()} {gender} (CBOW)---------\n")
            print(model_list[chosen_name].wv.similar_by_word(word, num_words))
            for entry in model_list[chosen_name].wv.similar_by_word(word, num_words):
                try:
                    word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                    word_embeddings[key].append(entry[0].lower())
                except Exception as error:
                    print(f"Could not write to file: {error}")
            word_file.write("\n----------------------------------------------\n")
        except Exception as error:
            print(f"no instance of {word} found\n\n")

In [ ]:
word_embeddings = {}
# SYMPTOMS
find_embeddings(word_embeddings, symptoms,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, symptoms,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, symptoms,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, symptoms,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, symptoms,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, symptoms,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, symptoms,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, symptoms,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)

# CONDITIONS
find_embeddings(word_embeddings, conditions,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, conditions,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, conditions,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, conditions,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, conditions,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, conditions,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, conditions,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, conditions,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# INTERPERSONAL
find_embeddings(word_embeddings, interpersonal,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, interpersonal,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, interpersonal,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, interpersonal,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, interpersonal,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, interpersonal,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, interpersonal,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, interpersonal,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# BEHAVIOUR
find_embeddings(word_embeddings, behaviour,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, behaviour,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, behaviour,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, behaviour,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, behaviour,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, behaviour,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, behaviour,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, behaviour,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# CLINICAL
find_embeddings(word_embeddings, clinical,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, clinical,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, clinical,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, clinical,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, clinical,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, clinical,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, clinical,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, clinical,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# STEREOTYPES
find_embeddings(word_embeddings, stereotypes,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, stereotypes,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, stereotypes,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, stereotypes,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, stereotypes,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, stereotypes,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, stereotypes,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, stereotypes,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# SUBJECTIVE
find_embeddings(word_embeddings, subjective,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, subjective,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, subjective,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, subjective,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, subjective,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, subjective,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, subjective,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, subjective,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)
# ECONOMICS
find_embeddings(word_embeddings, economics,
                white_fem_cbow_models,
                white_fem_cbow_model_chosen,
                "white", "FEMALE", 25)
find_embeddings(word_embeddings, economics,
                white_mal_cbow_models,
                white_mal_cbow_model_chosen,
                "white", "MALE", 25)
find_embeddings(word_embeddings, economics,
                black_fem_cbow_models,
                black_fem_cbow_model_chosen,
                "black", "FEMALE", 25)
find_embeddings(word_embeddings, economics,
                black_mal_cbow_models,
                black_mal_cbow_model_chosen,
                "black", "MALE", 25)
find_embeddings(word_embeddings, economics,
                mixed_fem_cbow_models,
                mixed_fem_cbow_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings(word_embeddings, economics,
                mixed_mal_cbow_models,
                mixed_mal_cbow_model_chosen,
                "mixed", "MALE", 25)
find_embeddings(word_embeddings, economics,
                asian_fem_cbow_models,
                asian_fem_cbow_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings(word_embeddings, economics,
                asian_mal_cbow_models,
                asian_mal_cbow_model_chosen,
                "asian", "MALE", 25)

print(f"LENGTH OF DICTIONARY: {len(word_embeddings.keys())}")
embeddings_file = open(f"embeddings_final/word_embeddings_25.txt", "w")
embeddings_file.write(json.dumps(word_embeddings, indent=2))

## Concordances

In [ ]:
def find_concs(text, word_list, ethnic_group):
    for word in word_list:
        curr_list = text.concordance_list(word)
        word_file = open(f"concordances/{ethnic_group}_{word}.txt", "a")
        for entry in curr_list:
            str_entry = " ".join(entry.left) + f" {word} " + " ".join(entry.right)
            word_file.write(str_entry)
            word_file.write("\n\n\n")
        word_file.close()

In [ ]:
# concordances
from nltk.text import Text
from nltk.corpus import gutenberg
nltk.download('gutenberg')
print(white_data.columns)
white_summaries = []
for i in range(white_data.shape[0]):
    toks = (" ".join([t.lower() for t in white_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    white_summaries.extend(toks)
white_text = Text(white_summaries)
find_concs(white_text, symptoms, "white")
find_concs(white_text, subjective, "white")
find_concs(white_text, stereotypes, "white")
find_concs(white_text, interpersonal, "white")
find_concs(white_text, economics, "white")
find_concs(white_text, conditions, "white")
find_concs(white_text, clinical, "white")
find_concs(white_text, behaviour, "white")


black_summaries = []
for i in range(black_data.shape[0]):
    toks = (" ".join([t.lower() for t in black_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    black_summaries.extend(toks)
black_text = Text(black_summaries)
find_concs(black_text, symptoms, "black")
find_concs(black_text, subjective, "black")
find_concs(black_text, stereotypes, "black")
find_concs(black_text, interpersonal, "black")
find_concs(black_text, economics, "black")
find_concs(black_text, conditions, "black")
find_concs(black_text, clinical, "black")
find_concs(black_text, behaviour, "black")

asian_summaries = []
for i in range(asian_data.shape[0]):
    toks = (" ".join([t.lower() for t in asian_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    asian_summaries.extend(toks)
asian_text = Text(asian_summaries)
find_concs(asian_text, symptoms, "asian")
find_concs(asian_text, subjective, "asian")
find_concs(asian_text, stereotypes, "asian")
find_concs(asian_text, interpersonal, "asian")
find_concs(asian_text, economics, "asian")
find_concs(asian_text, conditions, "asian")
find_concs(asian_text, clinical, "asian")
find_concs(asian_text, behaviour, "asian")

mixed_summaries = []
for i in range(mixed_data.shape[0]):
    toks = (" ".join([t.lower() for t in mixed_data['tokens'].iloc[i]])).split(" ")
    # print(toks[:10])
    mixed_summaries.extend(toks)
mixed_text = Text(mixed_summaries)
find_concs(mixed_text, symptoms, "mixed")
find_concs(mixed_text, subjective, "mixed")
find_concs(mixed_text, stereotypes, "mixed")
find_concs(mixed_text, interpersonal, "mixed")
find_concs(mixed_text, economics, "mixed")
find_concs(mixed_text, conditions, "mixed")
find_concs(mixed_text, clinical, "mixed")
find_concs(mixed_text, behaviour, "mixed")

In [ ]:
!tar czf embeddings_v3.tar.gz embeddings_final/*.txt

## Embedding Calculations---SkipGram

In [27]:
def find_embeddings_skip(word_embeddings, word_list, model_list, chosen_name, ethnic_group, gender, num_words):
    for word in word_list:
        word_file = open(f"embeddings_final/{word}{num_words}.txt", "a")
        try:
            key = f"{word}_{ethnic_group}_{gender.lower()}_skip"
            word_embeddings[key] = []
            word_file.write(f"---------{ethnic_group.upper()} {gender} (SKIP)---------\n")
            print(model_list[chosen_name].wv.similar_by_word(word, num_words))
            for entry in model_list[chosen_name].wv.similar_by_word(word, num_words):
                try:
                    word_file.write(f"\nWORD: {entry[0].lower()}\tSCORE: {entry[1]}")
                    word_embeddings[key].append(entry[0].lower())
                except Exception as error:
                    print(f"Could not write to file: {error}")
            word_file.write("\n----------------------------------------------\n")
        except Exception as error:
            print(f"no instance of {word} found\n\n")

In [28]:
word_embeddings_skip = {}
# SYMPTOMS
find_embeddings_skip(word_embeddings_skip, symptoms,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, symptoms,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)

# CONDITIONS
find_embeddings_skip(word_embeddings_skip, conditions,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, conditions,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# INTERPERSONAL
find_embeddings_skip(word_embeddings_skip, interpersonal,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, interpersonal,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# BEHAVIOUR
find_embeddings_skip(word_embeddings_skip, behaviour,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, behaviour,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# CLINICAL
find_embeddings_skip(word_embeddings_skip, clinical,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, clinical,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# STEREOTYPES
find_embeddings_skip(word_embeddings_skip, stereotypes,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, stereotypes,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# SUBJECTIVE
find_embeddings_skip(word_embeddings_skip, subjective,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, subjective,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)
# ECONOMICS
find_embeddings_skip(word_embeddings_skip, economics,
                white_fem_skip_models,
                white_fem_skip_model_chosen,
                "white", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                white_mal_skip_models,
                white_mal_skip_model_chosen,
                "white", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                black_fem_skip_models,
                black_fem_skip_model_chosen,
                "black", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                black_mal_skip_models,
                black_mal_skip_model_chosen,
                "black", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                mixed_fem_skip_models,
                mixed_fem_skip_model_chosen,
                "mixed", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                mixed_mal_skip_models,
                mixed_mal_skip_model_chosen,
                "mixed", "MALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                asian_fem_skip_models,
                asian_fem_skip_model_chosen,
                "asian", "FEMALE", 25)
find_embeddings_skip(word_embeddings_skip, economics,
                asian_mal_skip_models,
                asian_mal_skip_model_chosen,
                "asian", "MALE", 25)

print(f"LENGTH OF DICTIONARY: {len(word_embeddings_skip.keys())}")
embeddings_file = open(f"embeddings_final/word_embeddings_skip_25.txt", "w")
embeddings_file.write(json.dumps(word_embeddings_skip, indent=2))

[('abnormality', 0.9870731234550476), ('thinking', 0.9844253659248352), ('negative', 0.9835936427116394), ('cognition', 0.982192873954773), ('hopelessness', 0.9809640049934387), ('fatigue', 0.9801580905914307), ('distressing', 0.979241669178009), ('guilt', 0.9777061939239502), ('content', 0.9764004945755005), ('perception', 0.9762948155403137), ('abnormal', 0.9747251272201538), ('Thoughts', 0.9745951294898987), ('fleeting', 0.971863865852356), ('occasional', 0.9709981679916382), ('feature', 0.970547616481781), ('hallucination', 0.9701992869377136), ('auditory', 0.9701176285743713), ('tearfulness', 0.9700037240982056), ('anger', 0.9684203267097473), ('critical', 0.9680790901184082), ('panic', 0.9677911996841431), ('affective', 0.9658251404762268), ('disturbance', 0.9652494192123413), ('occasionally', 0.9636414051055908), ('irritable', 0.9619916677474976)]
[('functional', 0.9738500118255615), ('severity', 0.9695042371749878), ('attribute', 0.9693677425384521), ('withdrawal', 0.9659011960

231222